# Acquiring Financial Contributions Data from OpenSecrets.org

In [1]:
import configparser
import pymongo
import requests
import json
import us

## Configuration

In [2]:
config = configparser.ConfigParser()
config.read('../database-dev/auth/config.ini')
FEC_KEY = config.get('fec', 'FEC_API_KEY')
OPENSECRETS_KEY = config.get('opensecrets', 'OPEN_SECRETS_API_KEY')
OPENSECRETS_ROOT = config.get('opensecrets', 'OPEN_SECRETS_ROOT')
MONGO_LOCAL = config.get('mongodb', 'MONGO_LOCAL')
MONGO_DB = config.get('mongodb', 'MONGO_DB')
client = pymongo.MongoClient(MONGO_LOCAL)
db = client.get_database(MONGO_DB)

In [3]:
collection = db['reps']

In [4]:
states = [ state.abbr for state in us.states.STATES ]

In [16]:
projection_dict = {
    '_id': 1,
    'crp_id': 1,
    'fec_id': 1,
}
results = collection.find({'in_office': True, 'state': {'$in': states}}, projection_dict)
reps = [ rep for rep in results ]

In [6]:
len(reps)

432

In [18]:
reps[1]

{'_id': 'A000055', 'crp_id': 'N00003028', 'fec_id': 'H6AL04098'}

### Notes:
- Several reps were missing ```crp_id``` (Center for Responsive Politics IDs).
- Cross-checking reps by state in the OpenSecrets.org database, the entries of those reps do not exist in the database.

In [7]:
missing = []
for rep in reps:
    if (rep['crp_id'] == None) or (rep['crp_id'] == ''):
        missing.append(rep)

In [8]:
len(missing)

57

## Acquisition of Data

In [ ]:
def get_contributions(crp_id):
    url = OPENSECRETS_ROOT
    headers = {
        'method': 'candSector',
        'cid': crp_id,
        'output': 'json',
        'apikey': OPENSECRETS_KEY
    }
    r = requests.get(url, headers=headers)
    result = r.json()['response']['sectors']['sector']
    
    sectors = []
    for sector in result:
        contribution_dict = {
            'sector': sector['@attributes']['sector_name'],
            'individual': sector['@attributes']['indivs'],
            'pacs': sector['@attributes']['pacs'],
            'total': sector['@attributes']['total']
        }
        sectors.append(contribution_dict)
    
    return sectors

In [ ]:
for rep in reps:
    crp_id = rep['crp_id']
    try:
        rep['sectors'] = get_contributions(crp_id)
    except:
        rep['sectors'] = None